In [1]:
import pandas as pd

In [68]:
# df = pd.read_excel("Вопросы и ответы.xlsx")
# df.columns = df.iloc[0]
# df = df.iloc[1:]
# df.head()

KeyboardInterrupt: 

In [69]:
df = pd.read_csv("after_preprocessing.csv")
df.head()

,Unnamed: 0,Категория,Краткое описание,Описание,Решение,Аналитика 1,Аналитика 2,Аналитика 3
0,1,7,Без СППР. FW: 2.0 _АО_ нет графы количества,В авансовых отчетах программы XС X.X нет графы...,добрый день! Доработка функционала XC ERP X.X...,"Казначейство, взаиморасчеты",Расчеты с персоналом,Авансовые отчеты
1,2,7,Без СППР. 2023 ЗИ - 1.2 ФУНКЦИОНАЛ 1С ЕРП2. ...,Система «срезает» часть номера ЗРС при отражен...,Доработка функционала возможна в рамках запрос...,"Казначейство, взаиморасчеты",Казначейство,Заявки на оплату
2,3,7,Без СППР. Настройки для отчета по дебиторско...,Добрый день! Отправляю обращение проектной ко...,Добрый день. а. В системе созданы отчеты по де...,"Казначейство, взаиморасчеты",Взаиморасчеты,Отчеты
3,4,7,Без СППР. 1.2 ФУНКЦИОНАЛ 1С ЕРП2: в дополн...,"Добрый день, Признак ``Списать на расходы`` ...",Доработка функционала возможна в рамках запрос...,"Казначейство, взаиморасчеты",Казначейство,Платежные документы
4,5,7,Без СППР. 1.2 ФУНКЦИОНАЛ 1С ЕРП2. Актуально...,XX.XX.XXXX SDXXXXXXXX требуется уточнение - во...,"Добрый день, Наталья Сергеевна Доработка функ...","Казначейство, взаиморасчеты",Казначейство,Платежные документы


In [70]:
df = df[["Описание", "Решение"]]
df.head()

,Описание,Решение
0,В авансовых отчетах программы XС X.X нет графы...,добрый день! Доработка функционала XC ERP X.X...
1,Система «срезает» часть номера ЗРС при отражен...,Доработка функционала возможна в рамках запрос...
2,Добрый день! Отправляю обращение проектной ко...,Добрый день. а. В системе созданы отчеты по де...
3,"Добрый день, Признак ``Списать на расходы`` ...",Доработка функционала возможна в рамках запрос...
4,XX.XX.XXXX SDXXXXXXXX требуется уточнение - во...,"Добрый день, Наталья Сергеевна Доработка функ..."


In [71]:
def concatenate_text(examples):
    return {
        "text":
        examples["Описание"]
        + " \n " +
        examples["Решение"]
    }

In [72]:
# !pip install datasets

In [73]:
from datasets import Dataset

In [74]:
df = df[:100]

In [75]:
dt = Dataset.from_pandas(df)
dt

Dataset({
    features: ['Описание', 'Решение'],
    num_rows: 100
})

In [76]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [77]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [78]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [79]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [80]:
dt = dt.map(concatenate_text)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [81]:
embeddings_dataset = dt.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [82]:
!pip install faiss-cpu
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-cpu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [83]:
import faiss
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Описание', 'Решение', 'text', 'embeddings'],
    num_rows: 100
})

Составили эмбеддинги, переходим к токенизации?

In [89]:
request = "Добрый день, такая проблема: документы не скачиваются. Что делать?"

In [90]:
request_embedding = get_embeddings([request]).cpu().detach().numpy()

In [91]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", request_embedding, k=5
)

In [92]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)


In [93]:
for _, row in samples_df.iterrows():
    print(row)
    print("=" * 50)
    print()

Описание      Добрый день, коллеги. Вы пишите, что документы...
Решение       Здравствуйте!  Вопрос насколько знаю не актуал...
text          Добрый день, коллеги. Вы пишите, что документы...
embeddings    [-0.04505389928817749, -0.349048912525177, -0....
scores                                                15.928876
Name: 4, dtype: object

Описание      Добрый день! Для создания резерва под обесцене...
Решение       Добрый день! Для анализа номенклатуры без движ...
text          Добрый день! Для создания резерва под обесцене...
embeddings    [-0.010267294943332672, -0.20020823180675507, ...
scores                                                15.786793
Name: 3, dtype: object

Описание            Добрый день.       Необходимо доработать...
Решение       Доработана связка объекта строительства и объе...
text                Добрый день.       Необходимо доработать...
embeddings    [0.0145293353125453, -0.2552527189254761, -0.3...
scores                                                14